## Connect to the AIMMdb server

In [ ]:
from tiled.client import from_uri

In [ ]:
c = from_uri("https://aimm.lbl.gov/api")
c # Only 4 datasets can be seen while requesting data with public access

In [ ]:
# Public datasets should let public users see the data
c['nmc_sim']

In [ ]:
# To have access to private datasets, authorized users must login first
c.login()

In [ ]:
list(c) # Once user has logged in, more datasets are available

## Find data and metadata

In [ ]:
#Total of samples in NMC
c['nmc']

In [ ]:
df = c['nmc']['WwFMvg92n23'].read() # We can read the data in any child node by calling its uid
df # At this point, the user obtains a pandas dataframe, a more familiar data structure

In [ ]:
c['nmc']['WwFMvg92n23'].metadata # We can learn more about this data by reading the metadata in this node

In [ ]:
# Iterate through a collection of data. You can pick from these options: .items(), .keys(), .values()
c['nmc'].values()[3]

In [ ]:
for key, value in c['nmc'].items():
    # This would be a typical approach in python while attempting to filter data based on some information in the metadata.
    # This could involve multiple calls to the server
    # A more optimal approach is shown later in the document involving search()
    if value.metadata['element']['symbol'] == 'O' and value.metadata['element']['edge'] == 'K':
        print(f"{key}: {value}")

In [ ]:
# Using distinct in a dataset allows you to find all the possible values that a specific key could take
c['nmc'].distinct("facility.name", counts=True)

In [ ]:
from tiled.queries import Key

In [ ]:
# You can request the server to return some data of interest based on information in the metadata.
# This allows to optimize the call from the client and inmprove the perfgormance.
# Nested search calls are allowed. The client will not send multiple queries to the server. 
# Instead, it recognizes the chain of search calls and builds one query with the correct filters.
c['nmc'].search(Key("element.symbol") == "Co").search(Key("facility.name") == "NSLS-II")

In [ ]:
element_list = ["Co", "Mn", "Ni"]

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# We can use the search() method  to filter out experiments with very specific features in a dataset. 
# In this case, we ask tiled to find experiments using Co, Mn and Ni elements that are part of od the NCM622 sample,
# have a charge cycle value of 1, have a charge voltage value of 4.8 and were generated by ALS 
als_dfs = {}
als_metadata = {}
for element in  element_list:
    als_data = c['nmc'].search(Key("sample.name") == "NCM622").search(
        Key("element.symbol") == element).search(Key("charge.cycle") == 1).search(
        Key("charge.voltage") == 4.8).search(Key("facility.name") == "ALS").values().first()
    als_dfs[element] = als_data.read()
    als_metadata[element] = als_data.metadata

counter = 0
fig_als, ax_als = plt.subplots(1,3, figsize=(15, 4))
for key, df in als_dfs.items():
    if key == "Mn":
        ax_als[counter].plot(als_dfs[key]["energy"], als_dfs[key]["normfluor"], label=als_metadata[key]["element"]["symbol"])
        ax_als[counter].set(xlabel='energy', ylabel='normfluor')
    else:
        ax_als[counter].plot(als_dfs[key]["energy"], als_dfs[key]["mu_tfy"], label=als_metadata[key]["element"]["symbol"])
        ax_als[counter].set(xlabel='energy', ylabel='mu_tfy')
    ax_als[counter].legend()
    counter += 1
plt.show()

In [ ]:
# A similar search but for APS data
aps_dfs = {}
aps_metadata = {}
for element in  element_list:
    aps_data = c['nmc'].search(Key("sample.name") == "NCM622").search(
        Key("element.symbol") == element).search(Key("charge.cycle") == 1).search(
        Key("charge.voltage") == 4.8).search(Key("facility.name") == "APS").values().first()
    aps_dfs[element] = aps_data.read()
    aps_metadata[element] = aps_data.metadata

counter = 0
fig_aps, ax_aps = plt.subplots(1,3, figsize=(15, 4))
for key, df in aps_dfs.items():
    ax_aps[counter].plot(aps_dfs[key]["energy"], aps_dfs[key]["mutrans"], label=aps_metadata[key]["element"]["symbol"])
    ax_aps[counter].set(xlabel='energy', ylabel='mutrans')
    ax_aps[counter].legend()
    counter += 1
plt.show()

In [ ]:
# A similar search but for NSLS-II data
nslsii_dfs = {}
nslsii_metadata = {}
for element in  element_list:
    nslsii_data = c['nmc'].search(Key("sample.name") == "NCM622").search(
        Key("element.symbol") == element).search(Key("charge.cycle") == 1).search(
        Key("charge.voltage") == 4.8).search(Key("facility.name") == "NSLS-II").values().first()
    nslsii_dfs[element] = nslsii_data.read()
    nslsii_metadata[element] = nslsii_data.metadata
    
counter = 0
fig_nslsii, ax_nslsii = plt.subplots(1,3, figsize=(15, 4))
for key, df in nslsii_dfs.items():
    ax_nslsii[counter].plot(nslsii_dfs[key]["energy"], nslsii_dfs[key]["mutrans"], label=nslsii_metadata[key]["element"]["symbol"])
    ax_nslsii[counter].set(xlabel='energy', ylabel='mutrans')
    ax_nslsii[counter].legend()
    counter += 1
plt.show()

In [ ]:
# In a similar take, we can use this data to analyze a material with the same features across the three facilities

fig_element, ax_element = plt.subplots(1,3, figsize=(15, 4))

counter = 0
for key in nslsii_dfs.keys():
    if key == "Mn":
        ax_element[counter].plot(als_dfs[key]["energy"], als_dfs[key]["normfluor"], label="ALS")
    else:
        ax_element[counter].plot(als_dfs[key]["energy"], als_dfs[key]["mu_tfy"], label="ALS")
    
    ax_element[counter].plot(aps_dfs[key]["energy"], aps_dfs[key]["mutrans"], label="APS")
    ax_element[counter].plot(nslsii_dfs[key]["energy"], nslsii_dfs[key]["mutrans"], label="NSLS-II")
    ax_element[counter].set(title=key)
    
    ax_element[counter].legend()
    counter += 1
plt.show()

## Share data with collaboration

In [ ]:
import pandas as pd

In [ ]:
# Sometimes, the user might need to add new data to aimmdb. If the user has writing permissions they can follow the next steps

# Let's create some sample data first
data = {"node_id": [1, 2, 3],
       "probability": [0.2, 0.7, 0.1]}
df = pd.DataFrame(data)

metadata = {"dataset": "demo",
            "element": {"edge": "K", "symbol": "Ni"},
            "facility": "NSLS-II",
           }

In [ ]:
# We might need to save it in a dataset apart from the existing one.
# This would require to start a new dataset by creating a new container with tiled
c.create_container(key='demo')

In [ ]:
# The new data is written inside the new container
df_node = c['demo'].write_dataframe(df, metadata=metadata)
df_node

In [ ]:
# We can confirm that the new data is in aimmdb now
node_id = df_node.item["id"]
c['demo'][node_id]

In [ ]:
# Also, the new dataset should be visible from the root node
list(c)

In [ ]:
# If necessary, users can download their data. Most common file formats are available
c['demo'][node_id].export("demo.csv")
c['demo'][node_id].export("demo.json")
c['demo'][node_id].export("demo.txt")

In [ ]:
# If there is data that is not needed anymore, we can delete it.
# In this case, we are removing the demo data that we just created and leaving aimmdb the way we found it
c['demo'].delete(node_id)
c['demo']

In [ ]:
# We can delete the demo dataset, too.
c.delete('demo')
list(c)